In [1]:
# Importing librairies
import os
import shutil
import random
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from pathlib import Path
import yaml
from ultralytics import YOLO


In [2]:
# Paths
xml_folder = '../data/annotations'       
image_folder = '../data/images'          
train_output_folder = '../data/train/labels' 
train_folder = '../data/train/images'  
val_output_folder = '../data/val/labels'  
val_folder = '../data/val/images'       

# Creating output folders if they don't exist
Path(train_output_folder).mkdir(parents=True, exist_ok=True)
Path(train_folder).mkdir(parents=True, exist_ok=True)
Path(val_output_folder).mkdir(parents=True, exist_ok=True)
Path(val_folder).mkdir(parents=True, exist_ok=True)

# We will go through all the XML files
xml_files = [f for f in os.listdir(xml_folder) if f.endswith('.xml')]

# Splitting XML files into train and validation sets (80% train, 20% validation)
train_files = xml_files[:int(0.8 * len(xml_files))]  # 80% for training
val_files = xml_files[int(0.8 * len(xml_files)):]    # 20% for validation


def xml_to_yolo(xml_file, image_name, output_folder):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Opening corresponding .txt file to save YOLO annotations
    txt_file_path = os.path.join(output_folder, image_name.replace('.png', '.txt'))
    with open(txt_file_path, 'w') as f:
        for obj in root.findall('object'):
            # Get the object class name
            class_name = obj.find('name').text

            class_id = 0  # 0 is used for single class
            
            # Getting bounding box coordinates
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            
            # Convertting to YOLO format
            image_path = os.path.join(image_folder, image_name)
            image_width, image_height = 640, 640 
            
            # Calculating YOLO format coordinates
            x_center = (xmin + xmax) / 2 / image_width
            y_center = (ymin + ymax) / 2 / image_height
            width = (xmax - xmin) / image_width
            height = (ymax - ymin) / image_height
            
            # Writing to the YOLO annotation file
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

In [3]:
# Function to process images and annotations
def process_images(xml_files, output_folder, image_folder, label_folder):
    for xml_file in xml_files:
        image_name = xml_file.replace('.xml', '.png')  # Assuming image extension is .png
        image_path = os.path.join(image_folder, image_name)

        if os.path.exists(image_path):
            # Path to the XML file
            xml_file_path = os.path.join(xml_folder, xml_file)
            
            # Converting XML to YOLO format
            xml_to_yolo(xml_file_path, image_name, label_folder)
            
            # Copying image to the appropriate folder (train or validation)
            shutil.copy(image_path, output_folder)
        else:
            print(f"Warning: Image {image_name} not found, skipping.")

# Processing training data
print("Processing training data...")
process_images(train_files, train_folder, image_folder, train_output_folder)

# Processing validation data
print("Processing validation data...")
process_images(val_files, val_folder, image_folder, val_output_folder)

print("Dataset preparation complete.")


Processing training data...
Processing validation data...
Dataset preparation complete.


In [4]:
import os

# Define the path for the data.yaml file in the project root
data_yaml_path = '../data.yaml'

# Define the content of the data.yaml file
data_yaml = """
train: ALPR/data/images  # Path to training images
val: ALPR/data/images      # Path to validation images

nc: 1                       # Number of classes (in your case, 1 for license plates or other)

names: ['license_plate']     # Name of your class (customize as needed)

"""

# Save the YAML file to the project root directory
with open(data_yaml_path, 'w') as f:
    f.write(data_yaml)

print(f"data.yaml file created at {data_yaml_path}.")


data.yaml file created at ../data.yaml.


In [ ]:
# Initializing the YOLOv8 model
model = YOLO('yolov8s.pt')  # Loading the YOLOv8 model

# Training with YOLOv8
model.train(
    data=data_yaml_path,       # Path to the YAML file
    imgsz=416,                 # Image size
    batch=32,                  # Bbatch size
    epochs=5,                  # Number of epochs. I am setting it to 5 so it does not take forever to train
    cache='ram',               # We will use RAM caching for faster data loading
    amp=True,                  # Automatic mixed precision for faster training
    workers=4,                 # Number of workers for data loading
    device='cpu'               # Because my computer uses CPU and not GPU
)


Ultralytics 8.3.32  Python-3.11.7 torch-2.3.0+cpu CPU (12th Gen Intel Core(TM) i7-1265U)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=../data.yaml, epochs=5, time=None, patience=100, batch=32, imgsz=416, save=True, save_period=-1, cache=ram, device=cpu, workers=4, project=None, name=train25, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, li

train: Scanning C:\Users\Eyram\Documents\GitHub\ALPR\data\labels.cache... 26 images, 401 backgrounds, 0 corrupt: 100%|██████████| 427/427 [00:00<?, ?it/s]


WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.1GB RAM): 100%|██████████| 427/427 [00:00<00:00, 591.47it/s]
val: Scanning C:\Users\Eyram\Documents\GitHub\ALPR\data\labels.cache... 26 images, 401 backgrounds, 0 corrupt: 100%|██████████| 427/427 [00:00<?, ?it/s]


WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        4/5         0G      3.763      6.338      2.268          1        416: 100%|██████████| 14/14 [02:55<00:00, 12.54s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:52<00:00,  7.49s/it]

                   all        427         26    0.00012      0.115   7.29e-05   1.89e-05



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/14 [00:00<?, ?it/s]

Using the Trained YOLO Model for Inference

In [ ]:
from ultralytics import YOLO
import cv2
import os

# Load the trained YOLO model
model = YOLO("runs/detect/train17/weights/best.pt")

# Let's set the paths
test_folder = "../data/test"  # Folder containing test images
save_folder = "runs/detect/test_results"  # Folder to save results
os.makedirs(save_folder, exist_ok=True)  # Creating folder if it doesn't exist

# Confidence and IoU thresholds
CONF_THRESHOLD = 0.5  # Minimum confidence score
IOU_THRESHOLD = 0.5   # IoU threshold for Non-Maximum Suppression (NMS)

# Now we will process each .png image in the test folder
for image_name in os.listdir(test_folder):
    if image_name.lower().endswith(".png"):
        image_path = os.path.join(test_folder, image_name)
        print(f"Processing: {image_name}")
        
        # We will now perform inference
        results = model(image_path, conf=CONF_THRESHOLD, iou=IOU_THRESHOLD)

        # Getting the detections (bounding boxes)
        detections = results[0].boxes
        if detections:
            print(f"Detections for {image_name}:")
            for box in detections:
                print(box.xywh)  # Printing bounding box details (x, y, w, h)
            
            # Visualizing results on the image
            annotated_image = results[0].plot()  # Annotating the image
            
            # Saveingthe annotated image with "detect" in the filename
            new_name = f"detect_{image_name}"
            save_path = os.path.join(save_folder, new_name)
            cv2.imwrite(save_path, annotated_image)
            print(f"Saved results to {save_path}")
            
            # Displaying the annotated image
            cv2.imshow(f"Detections - {new_name}", annotated_image)
            cv2.waitKey(0)  # Press any key to close the image
            cv2.destroyAllWindows()
        else:
            print(f"No detections for {image_name}.")


In [ ]:
# Let's calculate the metrics
metrics = model.val(data='../data.yaml')
print(metrics)  # Includes mAP, precision, recall
